## 基于GloVe词向量的文本分类
> 姓名：钟思祺  
>学号：21302010069

文档包含两个部分：
* 首先是**实验代码**部分。这部分详细介绍了本次实验的主要代码。
* 其次是**实验内容**。这部分在实现了基本代码的前提下，进行了四个补充实验来进一步探索如何提高模型性能。实验中达到的准确率最高为65.11%。


### PART1 实验代码

#### 首先，导入PyTorch和TorchText库，用于文本分类。

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import SST
from torchtext.data import Field, BucketIterator
from torchtext.vocab import GloVe

#### 定义了两个Field对象，用于处理数据的文本和标签。
* TEXT对象用于处理文本数据，设置了一些属性，包括将文本转换为小写、每个样本的固定长度为40，以及批次维度在第一维（batch_first=True）。
 > 这里选定`fix_length=40`是因为通过抽样输出数据集文本，发现绝大部分text长度都在40以下。通过多次实验发现fix_length=40在不降低准确率的前提下能明显缩短训练时长。
* LABEL对象用于处理标签数据，设置了sequential=False，表示标签是非序列化的。

In [2]:
# Field 用来处理数据，fix_length 是单组训练数据的长度
# LABEL 是对应数据的标签
TEXT = Field(lower=True, fix_length=40, batch_first=True)
LABEL = Field(sequential=False)

#### 加载SST数据集
SST是一种用于情感分析的数据集，包括训练集`train`、验证集`valid`和测试集`test`。
> 可选参数train_subtrees=True，*use all subtrees in the training set*,相当于增大了数据集

In [3]:
# Load SST dataset
train, valid, test = SST.splits(TEXT, LABEL) # SST.splits(TEXT, LABEL, train_subtrees=True) # 

#### 构建词汇表并加载了预训练的GloVe词向量。
这样，文本数据和标签数据就被处理成了可以用于模型训练的形式，词汇表中的词被映射为对应的索引。

In [4]:
# Build vocabulary and load GloVe vectors 构建词汇表
TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=100), max_size=20000, min_freq=10)
LABEL.build_vocab(train)

#### 这里定义了[ GloVe + Transfomer + Pooling + Classifier ]的模型
1. `nn.Embedding`用于将输入的词索引转换为对应的词向量表示。`self.embedding.weight.data.copy_`将预训练的词向量加载到词嵌入层的权重中,即用GloVe初始化embedding层权重。`self.embedding.weight.requires_grad = False`表示embedding权重不可学习。加这项是希望保持词向量的值不变，只使用它们进行特征提取而不更新它们
2. `nn.TransformerEncoder`构建了一个Transformer编码器。它接受词向量作为输入，并通过多层TransformerEncoderLayer对输入进行编码。`embedding_dim`表示词向量的维度，`nhead`表示注意力头的数量，`dim_feedforward`表示前馈神经网络的隐藏层维度
3. `self.pooling`用于对编码后的文本进行池化操作，将每个序列维度上的特征进行汇聚。这里使用了nn.AdaptiveMaxPool1d(1)，表示在序列维度上进行自适应最大池化，将每个序列维度的最大值提取出来。作为对照还使用过平均池化`nn.AdaptiveAvgPool1d(1)`，效果不如最大池化。
4. `self.classifier`定义了一个分类器，用于将池化后的特征映射到最终的类别概率。它由两个全连接层和一个ReLU激活函数组成。

In [5]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, num_classes):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.embedding.weight.data.copy_(TEXT.vocab.vectors)
        self.embedding.weight.requires_grad = False

        self.transformer_encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(embedding_dim, nhead=2, dim_feedforward=hidden_dim), num_layers)
        
        #self.pooling = nn.AdaptiveAvgPool1d(1)
        self.pooling = nn.AdaptiveMaxPool1d(1)
        
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        # print(x.shape)
        embedded = self.embedding(x)
        # print(embedded.shape)
        encoded = self.transformer_encoder(embedded)
        pooled = self.pooling(encoded.permute(0, 2, 1))
        pooled = pooled.view(pooled.size(0), -1)
        output = self.classifier(pooled)
        return output

#### 这里定义了[ Random + RNN + Pooling + Classifier ]的模型，大体框架和上面的模型相似

1. `nn.Embedding`用于将输入的词索引转换为对应的词向量表示。模型会在训练开始前，随机初始化词嵌入层的权重参数。这意味着每个单词的词向量将会被赋予一个随机的初始值。
2. `nn.RNN`构建了一个RNN（循环神经网络）层self.rnn，它接受词向量作为输入并通过循环连接处理序列数据。`embedding_dim`表示输入特征的维度,`hidden_dim`表示隐藏状态的维度，`num_layers`表示堆叠的RNN层的数量。
3. `self.pooling`,池化层。
4. `self.classifier`，分类器。

In [ ]:
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, num_classes):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.pooling = nn.AdaptiveAvgPool1d(1)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        embedded = self.embedding(x)
        rnn_output, _ = self.rnn(embedded)
        pooled = self.pooling(rnn_output.permute(0, 2, 1))
        pooled = pooled.view(pooled.size(0), -1)
        output = self.classifier(pooled)
        return output

#### 创建模型、优化器和损失函数
1. 使用TextClassificationModel类创建一个文本分类模型对象，并传入参数：len(TEXT.vocab)表示词汇表的大小，100表示`词向量的维度`，128表示`隐藏状态的维度`，2表示`encoder的数量`，len(LABEL.vocab)表示`类别的数量`
2. 使用`optim.Adam`函数创建一个Adam优化器对象，lr=0.001表示学习率设置为0.001。
3. 使用`nn.CrossEntropyLoss`函数创建一个交叉熵损失函数对象。交叉熵损失函数在多分类任务中常用，用于计算模型预测结果和真实标签之间的差异。

In [6]:
# Create model, optimizer, and loss function
model = TextClassificationModel(len(TEXT.vocab), 100, 128, 2, len(LABEL.vocab))
optimizer = optim.Adam(model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss()

#### 创建数据迭代器，用于对训练、验证和测试数据进行批量处理。
`train_iter`、`valid_iter`和`test_iter`分别表示训练集、验证集和测试集的数据迭代器。通过这些迭代器，可以逐批次地获取数据进行模型的训练、验证和测试。
`repeat=False`表示迭代器将在数据集上进行一次迭代，不会重复迭代。

In [7]:
# Create data iterators
batch_size = 16
train_iter, valid_iter, test_iter = BucketIterator.splits((train, valid, test), batch_size=batch_size, repeat=False)

#### 定义训练循环
* `optimizer.zero_grad()`在每个批次开始时，将优化器的梯度缓冲区清零。这是为了确保每个批次的梯度计算是独立的。
* `loss = criterion(predictions, batch.label)`计算预测结果与批次的标签之间的损失
* `loss.backward()`根据损失值，计算模型参数的梯度。
* `optimizer.step()`根据优化器的更新规则，更新模型的参数，以减小损失函数的值。

In [8]:
# Training loop
def train_model(model, iterator, optimizer, criterion):
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text)
        loss = criterion(predictions, batch.label)
        loss.backward()
        optimizer.step()

#### 验证循环，用于在验证集上评估模型的性能。
* `with torch.no_grad()`在评估过程中，使用torch.no_grad()上下文管理器，以禁用梯度计算。这样可以减少内存消耗和计算量。
* `return total_loss / len(iterator), correct / len(iterator.dataset)`计算平均损失和准确率，返回在整个验证集上的损失和准确率

In [9]:
# Validation loop
def evaluate_model(model, iterator, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text)
            loss = criterion(predictions, batch.label)
            total_loss += loss.item()
            predicted_labels = predictions.argmax(1)
            correct += (predicted_labels == batch.label).sum().item()
    return total_loss / len(iterator), correct / len(iterator.dataset)

#### 训练模型
* `N_EPOCHS = 16`定义了训练的总epoch数，这里设置为16，即模型将被训练16次。
* 调用前面定义的`train_model`函数,`evaluate_model`函数。
* 在每个epoch结束后，打印当前epoch的验证集损失和准确率。这样可以监控模型在训练过程中的性能，并观察模型是否出现过拟合或欠拟合的情况。

In [10]:
# Train the model
N_EPOCHS = 16
for epoch in range(N_EPOCHS):
    train_model(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate_model(model, valid_iter, criterion)
    print(f'Epoch: {epoch+1:02}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation Acc: {valid_acc*100:.2f}%')

Epoch: 01
	Validation Loss: 0.947 | Validation Acc: 57.04%
Epoch: 02
	Validation Loss: 0.879 | Validation Acc: 60.22%
Epoch: 03
	Validation Loss: 0.875 | Validation Acc: 61.31%
Epoch: 04
	Validation Loss: 0.873 | Validation Acc: 60.76%
Epoch: 05
	Validation Loss: 0.874 | Validation Acc: 61.13%
Epoch: 06
	Validation Loss: 0.862 | Validation Acc: 61.58%
Epoch: 07
	Validation Loss: 0.888 | Validation Acc: 59.58%
Epoch: 08
	Validation Loss: 0.885 | Validation Acc: 59.85%
Epoch: 09
	Validation Loss: 0.880 | Validation Acc: 60.31%
Epoch: 10
	Validation Loss: 0.912 | Validation Acc: 57.13%
Epoch: 11
	Validation Loss: 0.891 | Validation Acc: 59.85%
Epoch: 12
	Validation Loss: 0.933 | Validation Acc: 59.58%
Epoch: 13
	Validation Loss: 0.964 | Validation Acc: 60.49%
Epoch: 14
	Validation Loss: 0.938 | Validation Acc: 60.49%
Epoch: 15
	Validation Loss: 0.959 | Validation Acc: 61.04%
Epoch: 16
	Validation Loss: 0.990 | Validation Acc: 60.40%


#### 测试集上评估模型的性能
模型在测试集上进行评估，并打印测试集的损失和准确率。这样可以得到模型在未见过的数据上的表现，从而判断模型的泛化能力和性能。测试集的结果可以作为模型的最终评估指标，用于评估模型的整体性能。

In [11]:
# Test the model
test_loss, test_acc = evaluate_model(model, test_iter, criterion)
print(f'Test Loss: {test_loss:.4f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.9297 | Test Acc: 63.21%


### PART2 实验内容
>实验环境：pytorch-cpu(3.9.6)  
>torchtext==0.8.1


#### 一、排列多种模型组合，比较相同超参下的准确率、训练时间。
| 模型组合  | Test Acc |  训练时间   |Test Loss|
|-------|-----|--------|------|
| GloVe + RNN  |  59.23% |  2min2s |1.08|
| Random + RNN |  59.95% |  2min2s  |2.286|
| GloVe + Transformer   |  62.76% |  3min16s |  0.9199  |
| Random + Transformer   |  60.36% |  3min25s | 0.9507  |

从结果可以看出：
* Transformer相较于RNN，准确率更高，损失率更低，性能更好，但是相对训练时间较长。
* GloVe做embedding层的效果也要好于随机初始化。Random+RNN的Test Loss可以看出模型在训练过程中发生了过拟合现象。

#### 二、分类器前加一层Attention，探索对模型性能的影响
添加注意力层的定义；修改 forward 方法来应用注意力层

In [ ]:
import torch.nn.functional as F

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.att_weights = nn.Parameter(torch.Tensor(hidden_dim))
        #self.att_weights = nn.Parameter(torch.Tensor(hidden_dim, 1))

    def forward(self, x):
        att_scores = torch.matmul(x, self.att_weights.unsqueeze(1))
        #att_scores = torch.bmm(x, self.att_weights.unsqueeze(2)).squeeze(2)
        att_scores = F.softmax(att_scores, dim=1)
        weighted_input = x * att_scores
        output = torch.sum(weighted_input, dim=1)
        return output

结果如下：
| Epoch | Validation Loss | Validation Acc |
|-------|---|-----|
| 01 | 0.909 | 58.22%  |
| 02    | 0.886           | 60.85%         |
| 03    | 0.896           | 59.85%         |
| 04    | 0.890           | 59.58%         |
| 05    | 0.884           | 60.40%         |
| 06    | 0.923           | 59.95%         |
| 07    | 0.893           | 61.67%         |
| 08    | 0.872           | 60.76%         |
| 09    | 0.894           | 61.13%         |
| 10    | 0.895           | 60.49%         |
| 11    | 0.909           | 60.40%         |
| 12    | 0.890           | 59.95%         |
| 13    | 0.911           | 61.22%         |
| 14    | 0.946           | 60.85%         |
| 15    | 0.948           | 61.04%         |
| 16    | 0.912           | 60.31%         |

Test Loss: 0.8740 | Test Acc: 62.08%

由原理可知通过引入注意力层，模型能够自动学习输入序列中的重要部分，并为每个输入位置分配不同的注意力权重。
但是从结果上来看并没有起到明显的优化作用，可能是数据的多样性或复杂性不够。


#### 三、使用dropout技巧训练模型
使用dropout来训练模型，在模型的各个层之间添加`nn.Dropout`层。nn.Dropout层将在训练过程中随机将输入中的一些元素设置为0，以减少过拟合。

实验设置dropout的概率为0.5

结果如下：
| Epoch | Validation Loss|  Validation Acc |
|-----|--------|------|
|01|1.075 |40.33%|
|02| 0.955 | 55.68%|
|03|0.966 | 55.59%|
|04|0.931 |57.31%|
|05|0.920 |58.95%|
|06|0.916 |59.67%|
|07|0.904 |59.49%|
|08|0.912 |58.13%|
|09| 0.904 | 59.22%|
|10| 0.907 |  59.67%|
|11| 0.943 |  56.31%|
|12|0.914 | 59.40%|
|13| 0.909 |59.85%|
|14| 0.901 |  59.76%|
|15| 0.913 | 58.86%|
|16|0.901 | 59.67%|
Test Loss: 0.8868 | Test Acc: 60.68%

从训练结果来看，使用dropout训练的模型在验证集上的准确率逐渐提高，损失率逐渐下降，并在最后达到了59.67%的准确率，测试集上的准确率为60.68%。可见dropout能够有效减少过拟合现象。

#### 四、扩充学习数据
SST.splits(TEXT, LABEL, train_subtrees=True),将数据集中多个次构成的集合加入到训练样本中，相当于把训练数据扩充了70倍。探索通过增大学习的数据量来提高性能。
|Epoch	|Validation Loss|	Validation Acc|
|-----|--------|------|
|01|	0.830|	63.12%|
|02|	0.842	|62.85%|
|03|	0.872|	60.67%|
|04	|0.849	|63.22%|
|05	|0.864|	62.13%|
|06	|0.839	|62.49%|
|07	|0.842|	61.85%|
|08	|0.850	|64.03%|
|09	|0.871	|61.94%|
|10	|0.859|	63.31%|
|11	|0.866|	61.76%|
|12|	0.813	|64.85%|
|13	|0.894	|62.31%|
|14	|0.879|	62.58%|
|15	|0.888|	61.13%|
|16|	0.821	|64.03%|
Test Loss: 0.8121 | Test Acc: 65.11%

损失率和准确率都要明显优于数据量小时的得分。可见通过增大数据量可以让模型学习到更丰富的表征。此时的性能瓶颈可能在于模型的参数规模。